In [ ]:
class Encoder():
    def __call__(self, inputs):
        """
        Args:
          inputs: [batch_size, num_nodes, node_dim] tensor
        Returns:
          [batch_size, num_nodes, embedding_dim] tensor
        """
        pass
    
class Decoder():
    def __call__(self, encoder_output, context_node_id, visited_mask):
        """Looks up embedding for context node, then does masked attention over encoder outputs.
        
        Result is log probs over next nodes.
        
        Args:
          inputs: [batch_size, num_nodes, embedding_dim]
          ...
          
        Returns:
          [batch_size, num_nodes] tensor of log probs with masking applied.
        """
        pass

class BatchedState():
    def __init__():
        """
        Should mirror the StateTSP of "attention learn how to solve routing problems"
        """

class BatchedGraphs():
    def __init__(self, edge_costs):
        """
        Args:
          edge_costs: [batch_size, num_edges, 2]
        """
        self._edge_costs = edge_costs
        
        
class BatchedHeaps():
    def __init__(self, batched_graphs):
        pass
    
    def pop_batch(self):
        pass

In [1]:
# TODO: generate random edge costs
num_nodes_per_graph = 20
num_batches = 100
batch_size = 10

raw_graph = generate_random_data()
# C++
graphs = BatchedGraphs(edge_costs) # (TODO: convert from distance matrix to edges)
graphs.pre_sort_edges()

# Python / GPU
encoder = Encoder()
decoder = Decoder()   
encoder_output = Encoder(graphs)

# Existing state class of Attention project
state = StateTsp.init_state(graphs)

# C++ <-- python
cpp_node_heap <-- python_obj_node_heap # batch of tensors (several attributes)
cpp_graphs <-- graphs # batch of graphs 
# C++
queues = BatchedHeaps(cpp_graphs, cpp_state) # TODO: iterate over graph_size and create C++ Heap for each

# Python
while len(queues) > 0: # while queues not empty
    # C++
    current_nodes = queues.pop_batch() # inside nested for-loop over batch of c++ heap class
    
    # C++ -> Python -- small amount of memory
    # current_nodes:
    #  * prefixes / visited masks
    #  * context nodes (TODO: what does that mean?)
    
    # CPU -> GPU
    # * current_nodes
    # NOT node_hiddens (should already be on GPU, so no cost to send in inner loop)
    
    # Python / GPU
    action_log_probs = decoder(encoder_output, state)  # [batch, num_nodes] child log probs
    
    # ?? (probably python)
    special_actions = sample(action_log_probs)

    # GPU -> CPU
    # * special children [batch_size] of ints

    # ok to do on python CPU
    other_actions = get_other_actions(current_nodes, special_actions)
    
    # Convert actions to nodes
    # python or C++?
    
    # Python -> C++ -- no huge tensors
    # * batch of prefixes (though maybe this can live in C++ already)
    # * special children
    # * other children
    
    # C++
    # get batch of truncated gumbels
    # [batch] tensors of costs
    special_spanning_tree_costs = graphs.spanning_tree_cost(current_nodes.prefixes, special_children)
    # [batch] tensors of costs
    other_spanning_tree_costs = graphs.spanning_tree_cost(current_nodes.prefixes, other_children)
    
    # C++
    # missing: first node, next possible actions
    queues.batch_push(special_children, special_spanning_tree_costs, special_gumbels)
    queues.batch_push(other_children, other_spanning_tree_costs, other_gumbels)
   
    
    
    

NameError: name 'generate_random_data' is not defined

### STARTS HERE

In [1]:
import torch
import kruskals_cpp
import dirpg_cpp

In [2]:
from utils import load_model
from dirpg_tsp import dirpg as dirpg_m
from utils import utils_gumbel
import numpy as np
import copy

#### Let's import the inputs

In [3]:
def generate_random_distance_matrix(n):
    loc = torch.FloatTensor(n, 2).uniform_(0, 1)
    return (loc[:, None, :] - loc[None, :, :]).norm(p=2, dim=-1)

def generate_data(size,num_samples):
    return torch.FloatTensor(num_samples,size, 2).uniform_(0, 1)


In [4]:
class search_params:
    def __init__(self,
                 max_interactions=200,
                 alpha=1.0,
                 epsilon=2.0,
                 heuristic='mst',
                 independent_gumbel=False,
                 first_improvement=False,
                 dynamic_weighting = False,
                 dfs_like=False,
                 not_prune=False):        
        self.max_interactions = max_interactions
        self.first_improvement = first_improvement
        self.dynamic_weighting = dynamic_weighting
        self.independent_gumbel = independent_gumbel
        self.heuristic = heuristic
        self.dfs_like = dfs_like
        self.prune = not not_prune
        self.alpha=alpha
        self.epsilon = epsilon


class CPP_manager:
    def __init__(self, root_node, batch_size, graph_size):
        

        root_cpp_converter = dirpg_cpp.convert_python_to_cpp(*root_node.get())
        
        #print("cpp_converter PYTHON")
        #print(root_cpp_converter)
        
        self.batched_heaps = dirpg_cpp.BatchedHeaps(root_cpp_converter,
                                                    batch_size,
                                                    graph_size) 
        
        #self.objects_dict = {root_cpp_converter:batch_size}
        self.objects_list = [root_cpp_converter]
        self.batch_size = batch_size
        
    def pop_batch(self):
        
        parents, trajs = self.batched_heaps.pop_batch()
        
        """
        parents,to_remove= parents
        for converter in to_remove:
            self.objects_dict[converter] -= 1
            if self.objects_dict[converter] == 0:
                del self.objects_dict[converter]
        """
        return Node.create_node_from_cpp(parents), trajs
        
        
    def push_batch(self,node, to_avoid):
        cpp_converter = dirpg_cpp.convert_python_to_cpp(*node.get())
        #self.objects_dict.update({cpp_converter:copy.copy(self.batch_size)})
        self.objects_list.append(cpp_converter)
        self.batched_heaps.push_batch(cpp_converter, to_avoid)       


In [5]:
class Trajectory:
    main_queue = None
    def __init__(self):

        self.num_candidates = 0
        self.t_opt = None
        self.t_direct = None
    
        
    def set_trajectory(self, cpp_traj):
        self.num_candidates += 1
        if self.num_candidates == 1:  # if t_opt            
            self.t_opt = cpp_traj
            self.t_direct = cpp_traj
            self.main_queue.lower_bound[cpp_traj.idx] = cpp_traj.objective

        else:
            if cpp_traj.objective > self.t_direct.objective:

                self.t_direct = cpp_traj
                self.main_queue.lower_bound[cpp_traj.idx] = cpp_traj.objective

    def print_trajectory(self):
        print('--------  Trajectory  ---------')
        print('trajectories_list:  ', self.trajectories_list)
        print('t_opt:  ', self.t_opt)
        print('t_direct:  ', self.t_direct)
        print('objective:  ', self.objective)
        print('-------------------------------')
        
class Node:
    epsilon = 1.0
    alpha = 2.0  # 2: full walk on the MST
    dynamic_weighting = True
    graph_size = 20
    heuristic = ''

    mst_edges = None
    mst_val = None

    def __init__(self,
                 ids,  # ids of sample in the batch - maybe to remove
                 t,  # time-step (not necessarily the same t for each sample)
                 next_actions,  # mask: 1 available, 0 unavailable
                 not_visited,   # mask: 1 available, 0 unavailable (note that next_actions!=not_visited)
                 prefix,  # e.g.: [2,6,1,0,-1,-1,-1,-1]
                 lengths,
                 cur_coord=None,
                 done=False,
                 logprob_so_far=0,
                 bound_togo=None,
                 max_gumbel=None,
                 is_t_opt=True):  #  true: opt, false: direct


        self.ids = ids
        self.t = t
        self.next_actions = next_actions        
        self.not_visited = not_visited        
        self.prefix = prefix
        

        self.lengths = lengths
        self.cur_coord = cur_coord
        self.done = done
        self.logprob_so_far = logprob_so_far
        self.max_gumbel = max_gumbel
        self.is_t_opt = is_t_opt  
        self.cpp_conv = None
        
        if self.dynamic_weighting:
            self.alpha = 1 + self.alpha*(1-self.t/self.graph_size)

        

        self.bound_togo = bound_togo

        self.eps_reward = self.epsilon * (self.lengths + self.alpha * self.bound_togo)

        self.priority = self.get_priority()
        self.objective = self.priority
        self.upper_bound = self.priority  # self.get_upper_bound()

    def get_priority(self, alpha=2):
        return self.max_gumbel + self.eps_reward

    def get_priority_max_gumbel(self):
        return self.max_gumbel

    def get_upper_bound(self, city=None):
        return self.max_gumbel + self.epsilon * (self.lengths + 1.0*self.bound_togo) #self.lengths + self.bound_length_togo()


    def get_objective(self):
        """Computes the objective of the trajectory.
        Only used if a node is terminal.
        """
        return self.max_gumbel + self.epsilon * self.lengths
    
    def pack(self):
        """
        pack node to cpp API: float, bool, obj_adrs, int (row in the original batch)
             
        """         
        return [[self.priority[i].item(), self.is_t_opt[i].item(), self, i] for i in range(self.ids.size(0))]
    
    def get(self):
        return self.priority, self.is_t_opt, self.done, self.ids, self.t, self.next_actions, self.not_visited,\
               self.prefix, self.lengths, self.cur_coord, self.logprob_so_far, self.bound_togo, self.max_gumbel,\
               self
    
    def print(self):
        print(' -----------    Node     -----------')
        print('ids:  ', self.ids)
        #print('first_a:  ', self.first_a)
        print('prefix:  ', self.prefix)
        print('not visited:  ', self.not_visited)
        print('next_actions:  ', self.next_actions)
        print('t:  ', self.t)
        #print('distance matrix:')
        print('max_gumbel:  ', self.max_gumbel)
        print('epsilon:   ',self.epsilon)
        print('alpha:   ', self.alpha)
        print('alpha * len_togo:   ', self.alpha * self.bound_togo)
        print('eps_reward: ', self.eps_reward, self.eps_reward.size())
        print('priority: ', self.priority)
        print('objective: ', self.objective)
        #print('upper bound: ',self.upper_bound)
        print('lengths:  ', self.lengths)
        print('bound length togo: ', self.bound_togo)
        print('done:  ', self.done)
        print('logprob_so_far:  ', self.logprob_so_far)

        print('is_t_opt:  ', self.is_t_opt)
        print(' -------------------------------')
        
    @staticmethod
    def create_node_from_cpp(args):
        return Node(ids=args[0],
                    t=args[1],
                    next_actions=args[2].bool(),
                    not_visited=args[3].bool(),
                    prefix=args[4],
                    lengths=args[5],
                    cur_coord=args[6],
                    done=args[7].bool(),
                    logprob_so_far=args[8],
                    bound_togo=args[9],
                    max_gumbel=args[10],
                    is_t_opt=args[11].bool()) 
    @staticmethod
    def create_node_from_cpp_list(nodes_list):
        atts = [[] for _ in range(12)]
        for idx, node in enumerate(nodes_list):
            atts[0].append(node.python_node_object.ids[node.idx_original])
            atts[1].append(node.python_node_object.t[node.idx_original])
            atts[2].append(node.python_node_object.next_actions[node.idx_original])
            atts[3].append(node.python_node_object.not_visited[node.idx_original])
            atts[4].append(node.python_node_object.prefix[node.idx_original])
            atts[5].append(node.python_node_object.lengths[node.idx_original])
            atts[6].append(node.python_node_object.cur_coord[node.idx_original])
            atts[7].append(node.python_node_object.done[node.idx_original].bool())
            atts[8].append(node.python_node_object.logprob_so_far[node.idx_original])
            atts[9].append(node.python_node_object.bound_togo[node.idx_original])
            atts[10].append(node.python_node_object.max_gumbel[node.idx_original])
            atts[11].append(node.python_node_object.is_t_opt[node.idx_original])
        
        n = Node( ids=torch.stack(atts[0]),  
                  t=torch.stack(atts[1]), 
                  next_actions=torch.stack(atts[2]),
                  not_visited=torch.stack(atts[3]),
                  prefix=torch.stack(atts[4]),
                  lengths=torch.stack(atts[5]),
                  cur_coord=torch.stack(atts[6]),
                  done=torch.stack(atts[7]),
                  logprob_so_far=torch.stack(atts[8]),
                  bound_togo=torch.stack(atts[9]),
                  max_gumbel=torch.stack(atts[10]),
                  is_t_opt=torch.stack(atts[11]))
  
        return n
    
    

In [6]:
class PriorityQueue:
    def __init__(self,
                 init_state,
                 graphs,
                 epsilon,
                 search_params,
                 inference=False
                 ):

        special_action = init_state.prev_a
        not_visited = ~init_state.visited.bool()
        
        dummy_idx = special_action.repeat(1,init_state.loc.size(-1)).unsqueeze(1)        
        self.first_coord = torch.gather(init_state.loc, 1, dummy_idx)
        
        self.graph_size = graphs.size(1)
        self.batch_size = graphs.size(0)
        ##########################################
        #           global nodes parameters      #
        Node.alpha = search_params.alpha
        Node.epsilon = epsilon
        Node.dynamic_weighting = search_params.dynamic_weighting
        Node.heuristic = search_params.heuristic
        Node.graph_size = self.graph_size
        ##########################################
        #           TODO: sort edges, kruskals_cpp
        
        #self.mst_edges = mst.prim_pytorch(distance_mat)
        #self.mst_val = self.mst_edges.sum()
        
        device = not_visited.device
        prefix=-torch.ones(self.batch_size,self.graph_size, device=device, dtype=torch.long)
        #prefix = p.scatter_(-1,torch.zeros(self.batch_size, dtype=torch.long), special_action) #
        prefix[:,0] = special_action.squeeze(-1)
        root_node = Node(ids=init_state.ids,
                         t=init_state.i,
                         next_actions=not_visited,  # torch.tensor(not_visited),  # number of cities
                         not_visited=not_visited,
                         prefix=prefix,
                         lengths=torch.zeros(self.batch_size,1,device=device),
                         cur_coord=self.first_coord,
                         done=torch.zeros(self.batch_size,1,device=device, dtype=torch.bool),
                         logprob_so_far=torch.zeros(self.batch_size,1,device=device),
                         bound_togo=-4.0*torch.ones(self.batch_size,1,device=device), #self.mst_val,
                         max_gumbel=utils_gumbel.sample_gumbel(self.batch_size).unsqueeze(-1),
                         is_t_opt=torch.ones(self.batch_size,device=device, dtype=torch.bool))
      

        
        self.cpp_heaps = CPP_manager(root_node, self.batch_size, self.graph_size)
        Trajectory.main_queue = self
        self.batch_trajectories = [Trajectory() for _ in range(self.batch_size)]
        self.lower_bound = [-float('Inf') for _ in range(self.batch_size)]
        
        
        
        
        self.t_opt = None
        self.t_direct = None


        self.num_interactions = 0
        self.first_improvement = search_params.first_improvement
        self.max_interactions = search_params.max_interactions
        
        self.dynamic_weighting = search_params.dynamic_weighting
        self.inference = inference
        
        self.p = torch.ones(self.batch_size, dtype=torch.bool)*search_params.prune
        self.prune = torch.zeros(self.p.size(), dtype=torch.bool)

    def pop(self):
        
        current_node, trajs_list = self.cpp_heaps.pop_batch()
        #print('parents_list: ', len(parents_list), 'trajs_list: ', len(trajs_list))
        for t in trajs_list:
            self.batch_trajectories[t.idx].set_trajectory(t)

        
        """
        if self.num_interactions >= self.max_interactions:
            return 'break'

        if self.prune and self.lower_bound > parent.upper_bound:
            self.prune_count += 1
            return self.pop()
        """


        return current_node


    def expand(self, state, current_node, logprobs):
        print(current_node.is_t_opt)
        self.num_interactions += 1
        special_action = state.prev_a
        #visited = state.visited_.bool()
        not_visited = current_node.not_visited.scatter(-1, special_action.unsqueeze(-1), False) 
        is_done = torch.all(~not_visited.transpose(1,2), dim=1, keepdim=False)
        prefix = current_node.prefix.scatter(-1, state.i-1, special_action)        
        cur_coord = state.loc[current_node.ids, special_action]
        
        lengths = -state.lengths #.squeeze(-1)       

        lengths = torch.where(is_done,
                              lengths - (self.first_coord - cur_coord).norm(p=2, dim=-1),
                              lengths) # add distance to first node if it's complete trajectory
       
        

        lp_special = torch.gather(logprobs, -1, special_action)

        special_child = Node(
            ids=state.ids,
            t=current_node.t + 1,
            not_visited=not_visited, 
            prefix=prefix, 
            lengths=lengths, 
            cur_coord=cur_coord,
            done=is_done,
            logprob_so_far=current_node.logprob_so_far + lp_special,
            max_gumbel=current_node.max_gumbel,
            next_actions=not_visited,
            bound_togo=current_node.bound_togo,
            is_t_opt=current_node.is_t_opt)

        #to_prune = self.prune | (special_child.upper_bound < self.lower_bound)
        to_prune = self.prune
        
        #self.python_nodes_in_queue.update({special_child:self.batch_size})
        self.cpp_heaps.push_batch(special_child, to_prune.squeeze(0))
        """
        if self.prune and special_child.upper_bound < self.lower_bound:
            self.prune_count += 1
        else:
            heapq.heappush(self.queue, special_child)

        """
        # Sample the max gumbel for the non-chosen actions and create an "other
        # children" node if there are any alternatives left.

        other_actions = current_node.next_actions.scatter(-1, special_action.unsqueeze(-1), False)

        assert torch.all(other_actions.sum(-1) == current_node.next_actions.sum(-1) - 1),'other_actions.sum(-1): {} current_node.next_actions.sum(-1) - 1): {}'.format(other_actions.sum(-1),current_node.next_actions.sum(-1) - 1)

        if not self.inference:
            #num_entries_to_compute_per_sample = other_actions.sum(-1).view(-1)
          
            exp = torch.exp(logprobs)

            exp[~other_actions.squeeze(1)] = 0
            other_max_location = torch.log(exp.sum(-1)).unsqueeze(-1)

            other_max_gumbel = utils_gumbel.sample_truncated_gumbel(current_node.logprob_so_far + other_max_location,
                                                                    current_node.max_gumbel)
            
            
            ignore = (other_max_location == -np.inf).squeeze(-1)
            

            other_children = Node(
                ids=current_node.ids,
                t=current_node.t,
                not_visited=current_node.not_visited,
                prefix=current_node.prefix,
                lengths=current_node.lengths,
                cur_coord=current_node.cur_coord,
                done=current_node.done,
                logprob_so_far=current_node.logprob_so_far,
                max_gumbel=other_max_gumbel,
                next_actions=other_actions,
                bound_togo=current_node.bound_togo, # + self.mst_edges[special_action].sum(),
                is_t_opt=current_node.is_t_opt*False)

            """
            if self.prune and other_children.upper_bound < self.lower_bound:
                self.prune_count += 1
            else:
                heapq.heappush(self.queue, other_children)
            """ 
            
            #to_prune = self.prune | (other_children.upper_bound < self.lower_bound) | ignore
            to_prune = (self.prune | ignore)
            #cpp_conv = dirpg_cpp.NodeConverter(*other_children.get())
            #self.python_nodes_in_queue.update({special_child:self.batch_size})
            self.cpp_heaps.push_batch(other_children, to_prune)
            


In [7]:
def encode(x, dirpg):
    embeddings = dirpg.encoder(x, only_encoder=True)
    
    state = dirpg.encoder.problem.make_state(x)
    fixed = dirpg.encoder.precompute(embeddings)
    return state, fixed

def forward_and_update(batch_state, fixed, first_action=None):
    dirpg.decoder.eval()
    log_p, _ = dirpg.decoder(fixed, batch_state) #fixed[:batch.ids.size(0)]
    log_p = log_p[:, 0, :]
    _, selected = utils_gumbel.sample_gumbel_argmax(log_p)
    if first_action is not None:
        selected = torch.ones_like(selected) * first_action
        
    state = batch_state.update(selected, update_length=True)  # like env.step
    return log_p, state

In [8]:
batch_size = 10
graph_size = 8

x = torch.rand(batch_size, graph_size, 2) # (x,y) coordinates 

params = search_params()

model, _ = load_model('outputs/tsp_{}/jupyter_example/DirPG_20200506T134440/'.format(graph_size), epoch = 0) # 'pretrained/tsp_100/')
model.eval()  # Put in evaluation mode to not track gradients
dirpg = dirpg_m.DirPG(model, params)

def init_batched_priority_queue(x):    

    state, fixed = encode(x, dirpg)
    #print('*******  Before init state:  *******')
    #state.print_state()
    log_p, state = forward_and_update(state, fixed) #  forward decoder and env.step

    print('*******  After init state:  *******')
    state.print_state()

    print('*****  Fixed tensors - living only in python  ***** ')
    fixed.print_size() # living only in python and stays fixed during decoding 
    print()
    print(' ***** States tensors --> C++  ***** ')
    state.print_size() # 
    print()
    print(' ***** log_p tensors --> C++  ***** ')
    print(log_p.size())



    pq = PriorityQueue(init_state=state,
                       graphs=x,
                       epsilon=1.0,
                       search_params=params,
                       inference=False)
    return pq, state, fixed



  [*] Loading model from outputs/tsp_8/jupyter_example/DirPG_20200506T134440/epoch-0.pt


In [15]:
pq, state, fixed = init_batched_priority_queue(x)

*******  After init state:  *******
----------   state ----------
ids
tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]])
first_a
tensor([[7],
        [6],
        [1],
        [3],
        [1],
        [4],
        [3],
        [0],
        [0],
        [6]])
prev_a
tensor([[7],
        [6],
        [1],
        [3],
        [1],
        [4],
        [3],
        [0],
        [0],
        [6]])
visited_
tensor([[[0, 0, 0, 0, 0, 0, 0, 1]],

        [[0, 0, 0, 0, 0, 0, 1, 0]],

        [[0, 1, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0]],

        [[1, 0, 0, 0, 0, 0, 0, 0]],

        [[1, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 1, 0]]], dtype=torch.uint8)
lengths
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
    

In [16]:
with torch.no_grad():
    while pq.num_interactions < 200: ## interactions budget

        parents = pq.pop()
        prev_a = torch.gather(parents.prefix,-1,parents.t-1)
        #prev_a = torch.where(parents.t == parents.prefix.size(-1), state.first_a, prev_a)


        print(parents.prefix)
        visited_ = (~parents.next_actions).type(torch.uint8)

        state = state._replace(ids=parents.ids,
                        prev_a=prev_a,
                        visited_=visited_, # torch.uint8
                        lengths=-parents.lengths,
                        cur_coord=parents.cur_coord,
                        i=parents.t)

        log_p, state = forward_and_update(state, fixed)

        pq.expand(state,parents, log_p)


tensor([[ 7, -1, -1, -1, -1, -1, -1, -1],
        [ 6, -1, -1, -1, -1, -1, -1, -1],
        [ 1, -1, -1, -1, -1, -1, -1, -1],
        [ 3, -1, -1, -1, -1, -1, -1, -1],
        [ 1, -1, -1, -1, -1, -1, -1, -1],
        [ 4, -1, -1, -1, -1, -1, -1, -1],
        [ 3, -1, -1, -1, -1, -1, -1, -1],
        [ 0, -1, -1, -1, -1, -1, -1, -1],
        [ 0, -1, -1, -1, -1, -1, -1, -1],
        [ 6, -1, -1, -1, -1, -1, -1, -1]])
tensor([True, True, True, True, True, True, True, True, True, True])
tensor([[ 7,  1, -1, -1, -1, -1, -1, -1],
        [ 6,  2, -1, -1, -1, -1, -1, -1],
        [ 1,  7, -1, -1, -1, -1, -1, -1],
        [ 3,  5, -1, -1, -1, -1, -1, -1],
        [ 1,  7, -1, -1, -1, -1, -1, -1],
        [ 4,  7, -1, -1, -1, -1, -1, -1],
        [ 3,  1, -1, -1, -1, -1, -1, -1],
        [ 0,  3, -1, -1, -1, -1, -1, -1],
        [ 0,  7, -1, -1, -1, -1, -1, -1],
        [ 6,  0, -1, -1, -1, -1, -1, -1]])
tensor([True, True, True, True, True, True, True, True, True, True])
tensor([[ 7,  1,  0,

tensor([False, False, False, False, False, False, False, False, False, False])
tensor([[ 7,  4,  0, -1, -1, -1, -1, -1],
        [ 6,  7, -1, -1, -1, -1, -1, -1],
        [ 1,  2,  4, -1, -1, -1, -1, -1],
        [ 3,  6, -1, -1, -1, -1, -1, -1],
        [ 1,  5,  3, -1, -1, -1, -1, -1],
        [ 4,  7,  1,  0,  3,  5, -1, -1],
        [ 3,  4,  7, -1, -1, -1, -1, -1],
        [ 0,  2,  3,  1, -1, -1, -1, -1],
        [ 0,  5,  7,  2, -1, -1, -1, -1],
        [ 6,  7,  3, -1, -1, -1, -1, -1]])
tensor([False, False, False, False, False, False, False, False, False, False])
tensor([[ 7,  1,  0,  3,  4, -1, -1, -1],
        [ 6,  7,  5, -1, -1, -1, -1, -1],
        [ 1,  7,  2,  5,  6, -1, -1, -1],
        [ 3,  4,  0,  1, -1, -1, -1, -1],
        [ 1,  7, -1, -1, -1, -1, -1, -1],
        [ 4,  7,  1,  3,  0,  2, -1, -1],
        [ 3,  5, -1, -1, -1, -1, -1, -1],
        [ 0,  2,  3,  7,  5,  6,  1, -1],
        [ 0,  5,  6, -1, -1, -1, -1, -1],
        [ 6,  7,  4,  5,  2,  0, -1, -1]])


tensor([[ 7,  0,  2,  4, -1, -1, -1, -1],
        [ 6,  3,  7,  2, -1, -1, -1, -1],
        [ 1,  6,  4, -1, -1, -1, -1, -1],
        [ 3,  0,  1, -1, -1, -1, -1, -1],
        [ 1,  5,  3,  7, -1, -1, -1, -1],
        [ 4,  3,  1, -1, -1, -1, -1, -1],
        [ 3,  6,  1,  7,  4,  0, -1, -1],
        [ 0,  2,  3,  6,  5,  4,  7, -1],
        [ 0,  6,  5, -1, -1, -1, -1, -1],
        [ 6,  7,  1, -1, -1, -1, -1, -1]])
tensor([False, False, False, False, False, False, False, False, False, False])
tensor([[ 7,  5,  6, -1, -1, -1, -1, -1],
        [ 6,  7,  1,  5, -1, -1, -1, -1],
        [ 1,  2,  4,  6,  5, -1, -1, -1],
        [ 3,  2,  1,  0, -1, -1, -1, -1],
        [ 1,  3, -1, -1, -1, -1, -1, -1],
        [ 4,  7,  1,  3,  0, -1, -1, -1],
        [ 3,  6,  0,  1, -1, -1, -1, -1],
        [ 0,  2,  3,  1,  5,  6,  7, -1],
        [ 0,  5,  1, -1, -1, -1, -1, -1],
        [ 6,  7,  0,  4,  5, -1, -1, -1]])
tensor([False, False, False, False, False, False, False, False, False, False])


tensor([[ 7,  0,  5,  1, -1, -1, -1, -1],
        [ 6,  5,  3,  7,  0,  1, -1, -1],
        [ 1,  4,  5,  2, -1, -1, -1, -1],
        [ 3,  1,  2, -1, -1, -1, -1, -1],
        [ 1,  2,  6, -1, -1, -1, -1, -1],
        [ 4,  2,  1,  7, -1, -1, -1, -1],
        [ 3,  0, -1, -1, -1, -1, -1, -1],
        [ 0,  2,  7,  6, -1, -1, -1, -1],
        [ 0,  5,  7,  6,  2, -1, -1, -1],
        [ 6,  7,  2,  0,  4,  5, -1, -1]])
tensor([False, False, False, False, False, False, False, False, False, False])
tensor([[ 7,  0,  4, -1, -1, -1, -1, -1],
        [ 6,  3,  5,  1, -1, -1, -1, -1],
        [ 1,  5,  4,  2, -1, -1, -1, -1],
        [ 3,  7,  2,  4, -1, -1, -1, -1],
        [ 1,  3,  5, -1, -1, -1, -1, -1],
        [ 4,  3,  7,  1,  0, -1, -1, -1],
        [ 3,  6,  1, -1, -1, -1, -1, -1],
        [ 0,  2,  5,  7, -1, -1, -1, -1],
        [ 0,  7,  5,  2,  1,  6, -1, -1],
        [ 6,  7,  4,  0,  5,  2, -1, -1]])
tensor([False, False, False, False, False, False, False, False, False, False])


tensor([[ 7,  0,  5,  3,  2, -1, -1, -1],
        [ 6,  3,  7, -1, -1, -1, -1, -1],
        [ 1,  4,  5,  0, -1, -1, -1, -1],
        [ 3,  1,  2,  6, -1, -1, -1, -1],
        [ 1,  0,  6,  3, -1, -1, -1, -1],
        [ 4,  1,  7,  3,  0,  2,  6, -1],
        [ 3,  1,  6,  0,  4,  5, -1, -1],
        [ 0,  2,  1,  6, -1, -1, -1, -1],
        [ 0,  5,  2,  7, -1, -1, -1, -1],
        [ 6,  5,  7, -1, -1, -1, -1, -1]])
tensor([False, False, False, False, False, False, False, False, False, False])
tensor([[ 7,  1,  5, -1, -1, -1, -1, -1],
        [ 6,  3,  5, -1, -1, -1, -1, -1],
        [ 1,  5,  4,  6, -1, -1, -1, -1],
        [ 3,  6,  2,  5,  4,  1,  0, -1],
        [ 1,  2,  5, -1, -1, -1, -1, -1],
        [ 4,  5,  7, -1, -1, -1, -1, -1],
        [ 3,  2,  1,  0, -1, -1, -1, -1],
        [ 0,  2,  5,  6,  3, -1, -1, -1],
        [ 0,  2,  5, -1, -1, -1, -1, -1],
        [ 6,  7,  3,  2,  0,  4, -1, -1]])
tensor([False, False, False, False, False, False, False, False, False, False])


In [17]:
t_opt = torch.stack([tp.t_opt.actions for tp in pq.batch_trajectories])
t_direct = torch.stack([tp.t_direct.actions for tp in pq.batch_trajectories])

t_opt_costs = torch.tensor([tp.t_opt.cost for tp in pq.batch_trajectories])
t_direct_costs = torch.tensor([tp.t_direct.cost for tp in pq.batch_trajectories])

In [18]:
print(t_opt)
print(t_opt_costs)

tensor([[7, 1, 0, 4, 3, 2, 5, 6],
        [6, 2, 0, 7, 5, 1, 3, 4],
        [1, 7, 4, 2, 5, 0, 6, 3],
        [3, 5, 0, 4, 1, 6, 2, 7],
        [1, 7, 5, 3, 6, 2, 0, 4],
        [4, 7, 3, 1, 0, 6, 5, 2],
        [3, 1, 4, 6, 0, 2, 7, 5],
        [0, 3, 2, 6, 5, 7, 1, 4],
        [0, 7, 2, 4, 5, 3, 6, 1],
        [6, 0, 2, 5, 7, 3, 1, 4]])
tensor([-4.1697, -2.6829, -3.0020, -4.2574, -3.6829, -3.2750, -3.7542, -3.6266,
        -3.8893, -3.8187])


In [19]:
print(t_direct)
print(t_direct_costs)

tensor([[7, 1, 0, 4, 3, 2, 5, 6],
        [6, 2, 0, 7, 5, 1, 3, 4],
        [1, 7, 4, 2, 5, 0, 6, 3],
        [3, 5, 0, 4, 1, 6, 2, 7],
        [1, 7, 5, 3, 6, 2, 0, 4],
        [4, 7, 3, 1, 0, 6, 5, 2],
        [3, 1, 4, 6, 0, 2, 7, 5],
        [0, 3, 2, 6, 5, 7, 1, 4],
        [0, 7, 2, 4, 5, 3, 6, 1],
        [6, 0, 2, 5, 7, 3, 1, 4]])
tensor([-4.1697, -2.6829, -3.0020, -4.2574, -3.6829, -3.2750, -3.7542, -3.6266,
        -3.8893, -3.8187])


In [9]:
import random
def get_root(graphs,
             epsilon=1.0,
             search_params=params):
    
    state, fixed = encode(x, dirpg)
    log_p, init_state = forward_and_update(state, fixed)
    
    special_action = init_state.prev_a
    not_visited = ~init_state.visited.bool()

    dummy_idx = special_action.repeat(1,init_state.loc.size(-1)).unsqueeze(1)        
    first_coord = torch.gather(init_state.loc, 1, dummy_idx)

    graph_size = graphs.size(1)
    batch_size = graphs.size(0)
    ##########################################
    #           global nodes parameters      #
    Node.alpha = search_params.alpha
    Node.epsilon = epsilon
    Node.dynamic_weighting = search_params.dynamic_weighting
    Node.heuristic = search_params.heuristic
    Node.graph_size = graph_size
    ##########################################
    #           TODO: sort edges, kruskals_cpp

    #self.mst_edges = mst.prim_pytorch(distance_mat)
    #self.mst_val = self.mst_edges.sum()

    device = not_visited.device
    prefix=-torch.ones(batch_size,graph_size, device=device, dtype=torch.long)
    #prefix = p.scatter_(-1,torch.zeros(self.batch_size, dtype=torch.long), special_action) #
    prefix[:,0] = special_action.squeeze(-1)
    root_node = Node(ids=init_state.ids,
                     t=init_state.i,
                     next_actions=not_visited,  # torch.tensor(not_visited),  # number of cities
                     not_visited=not_visited,
                     prefix=prefix,
                     lengths=torch.zeros(batch_size,1,device=device),
                     cur_coord=first_coord,
                     done=torch.zeros(batch_size,1,device=device, dtype=torch.bool),
                     logprob_so_far=torch.zeros(batch_size,1,device=device),
                     bound_togo=-4.0*torch.rand(batch_size,1,device=device), #self.mst_val,
                     max_gumbel=utils_gumbel.sample_gumbel(batch_size).unsqueeze(-1),
                     is_t_opt=torch.ones(batch_size,device=device, dtype=torch.bool))
    return root_node

In [10]:
root_node = get_root(x)
cpp_obj = CPP_manager(root_node, batch_size, graph_size)

TypeError: convert_python_to_cpp(): incompatible function arguments. The following argument types are supported:
    1. (arg0: List[float], arg1: List[int], arg2: List[int], arg3: List[int], arg4: List[int], arg5: List[List[int]], arg6: List[List[int]], arg7: List[List[int]], arg8: List[float], arg9: List[List[float]], arg10: List[float], arg11: List[float], arg12: List[float], arg13: object) -> NodeConverter

Invoked with: tensor([[-3.7348],
        [ 0.1967],
        [-1.5242],
        [-2.4825],
        [-1.2709],
        [-1.4073],
        [-3.0137],
        [ 0.5720],
        [-2.8154],
        [-3.1725]]), tensor([True, True, True, True, True, True, True, True, True, True]), tensor([[False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False]]), tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]]), tensor

In [10]:
0x11171f5b0 0x11171f5b0
0x11171f3b0, 0x11171f3b0
0x11175ee70

In [14]:
node = get_root(x)

cpp_obj.push_batch(node, torch.zeros(10))


In [ ]:
cpp_obj.pop_batch()

In [10]:
#push
cpp_converter = dirpg_cpp.NodeConverter(new_python_node)
batched_heaps = dirpg_cpp.BatchedHeaps.push_batch(cpp_converter)

nodes_ = []
for i in range(5):
    
    nodes_.append(dirpg_cpp.HeapNode(cpp_converter.get_args_for_heap_node_constructor(i)))

In [1]:
import random
random.random()

0.9555630965592029

In [ ]:
dirpg_cpp.stack_heap_nodes_vec(nodes_)

In [ ]:
batched_heaps = dirpg_cpp.BatchedHeaps(cpp_converter)  #root_node.pack()
parents = batched_heaps.pop_batch()

In [37]:
def mask_minus_inf_logsumexp(x, idx):
    idx = ~idx
    x = x.scatter(-1, idx.long(), -float('Inf'))
    print(x)
    res = torch.logsumexp(x, -1)
    print(res)

In [95]:
def my_padded_logsumexp(x, idx):
    num_entries_to_compute_per_sample = idx.sum(-1).view(-1)
    non_even_concat_logprobs = x[idx]
    other_max_location = dirpg_cpp.padded_logsumexp(non_even_concat_logprobs,
                                                    num_entries_to_compute_per_sample)
    
    return other_max_location

In [ ]:
Timer unit: 1e-06 s

Total time: 0.00082 s
File: <ipython-input-95-8083cf90a2ba>
Function: my_padded_logsumexp at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def my_padded_logsumexp(x, idx):
     2         1        247.0    247.0     30.1      num_entries_to_compute_per_sample = idx.sum(-1).view(-1)
     3         1        185.0    185.0     22.6      non_even_concat_logprobs = x[idx]
     4         1          3.0      3.0      0.4      other_max_location = dirpg_cpp.padded_logsumexp(non_even_concat_logprobs,
     5         1        383.0    383.0     46.7                                                      num_entries_to_compute_per_sample)
     6                                               
     7         1          2.0      2.0      0.2      return other_max_location

In [101]:
def naive_logsumexp(x, idx):
    exp = torch.exp(x)
    exp[~idx] = 0
    other_max_location = torch.log(exp.sum(-1))
    return other_max_location


In [ ]:
Timer unit: 1e-06 s

Total time: 0.000516 s
File: <ipython-input-101-c0197651927e>
Function: naive_logsumexp at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def naive_logsumexp(x, idx):
     2         1        179.0    179.0     34.7      exp = torch.exp(x)
     3         1        290.0    290.0     56.2      exp[~idx] = 0
     4         1         46.0     46.0      8.9      other_max_location = torch.log(exp.sum(-1))
     5         1          1.0      1.0      0.2      return other_max_location

In [102]:
x = torch.tensor(range(5*4)).view(5,4).float()
print(x)
print('%%%%%%%%%')
idx = torch.tensor([[True, True, False,False],
                    [True, True, True, False],
                    [False, True, True, False],
                    [True, True, True, True],
                    [False, False, False,False]])

x[idx] = 0
x.sum(-1)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.],
        [16., 17., 18., 19.]])
%%%%%%%%%


tensor([ 5.,  7., 19.,  0., 70.])

In [103]:
#idx torch.Tensor([[0,2],[3,4]])

print(my_padded_logsumexp(x, idx))
print('--------------------')
print(naive_logsumexp(x, idx))

#x

tensor([[0.6931],
        [1.0986],
        [0.6931],
        [1.3863],
        [  -inf]])
--------------------
tensor([0.6931, 1.0986, 0.6931, 1.3863,   -inf])


In [104]:
%load_ext line_profiler

In [106]:
%lprun -f my_padded_logsumexp my_padded_logsumexp(x, idx)

In [100]:
torch.logsumexp(torch.tensor([2.,  3.]),0)

tensor(3.3133)

In [28]:
a = torch.ones(10,requires_grad=True)
a.requires_grad

TypeError: 'bool' object is not callable

In [73]:
torch.gather(x,-1,torch.tensor([[1],[1, 2]]))

ValueError: expected sequence of length 1 at dim 1 (got 2)

In [67]:
x[[0, 1], [[0, 2], [1, 2]]]

tensor([[0., 5.],
        [1., 5.]])

In [61]:
x.shape

torch.Size([2, 3])

In [53]:
x = torch.tensor(range(2*3)).view(2,3).float()
print(x)
idx = torch.tensor([[0, 2],
                    [3, 3]])

torch.logsumexp(x[[0, 2], [1, 2]])
#mask_minus_inf_logsumexp(x, idx)

#x.sum(-1)



tensor([[0., 1., 2.],
        [3., 4., 5.]])


IndexError: index 2 is out of bounds for dimension 0 with size 2

In [50]:
x

tensor([[ 0.,  5.,  2.],
        [ 3.,  4., 25.]])

In [46]:
x[[0,1], [[1, 2], [0,1]]]

tensor([[1., 5.],
        [0., 4.]])

In [30]:
!pip install line_profiler

     |████████████████████████████████| 45 kB 251 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for line-profiler: filename=line_profiler-3.0.2-cp37-cp37m-macosx_10_13_x86_64.whl size=52831 sha256=e867da2a8968e1e407c9038c3e8b5a2ea9f0b54eb1aefcacb40a8be10193be3b
  Stored in directory: /Users/guy_l/Library/Caches/pip/wheels/20/28/5c/69ef18a2dc224b4230d6b88e9c0721ec7d3bb425b71d3248ce
Successfully built line-profiler
You should consider upgrading via the '/Users/guy_l/Research/DirPG_TSP/dirpg/bin/python3 -m pip install --upgrade pip' command.


In [34]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [41]:
import scipy.sparse.csgraph as cs
import time

num_iter = 100
batch_size = 10
graph_size = 20


def run_cs_mst(num_iter, batch_size, graph_size):
    graphs = [generate_random_distance_matrix(graph_size) for _ in range(batch_size)]
    cost = torch.zeros(num_iter,batch_size)
    

    for i in range(num_iter):
        for j,graph in enumerate(graphs):        
            cost[i,j]=np.sum(cs.minimum_spanning_tree(graph))
            

In [ ]:
def convert_distance_matrix_to_batched_edges(num_iter, batch_size, graph_size):
    """distance_matrix: batch of distance matrices. size: [batch, n, n]
    returns weights_and_edges: in shape (batch_size, n * (n - 1) / 2, 3), where
    weights_and_edges[.][i] = [weight_i, node1_i, node2_i] for edge i."""
    
    graphs = [generate_random_distance_matrix(graph_size) for _ in range(batch_size)]
    graphs = torch.stack(graphs)
    
    batched_graphs = BatchedGraphs(graphs)
    batched_graphs.init_sort()
    cost = torch.zeros(num_iter,batch_size)
    
    for i in range(num_iter):
        cost[i,:] = batched_graphs.mst_costs()
    
    
        



In [42]:
%lprun -f run_cs_mst run_cs_mst(num_iter, batch_size, graph_size)

In [43]:

graphs = [generate_random_distance_matrix(graph_size) for _ in range(batch_size)]
torch.stack(graphs)

In [21]:
import torch
import dirpg_cpp
from utils import utils_gumbel

batch=3
graph_size=5
my_visited = torch.tensor([[0, 0, 0 ,0, 0],[0,1,1,0,1],[1,1,1,1,1]], dtype=torch.bool).unsqueeze(1)
print(my_visited)
x = torch.randn(batch,1,graph_size)
print(x)
#idx = torch.zeros(batch,1,1).scatter_(-1,torch.tensor([[]]))
print(my_visited.sum(-1))
x[my_visited]

tensor([[[False, False, False, False, False]],

        [[False,  True,  True, False,  True]],

        [[ True,  True,  True,  True,  True]]])
tensor([[[-0.1026,  0.2537, -0.3111,  0.3591,  1.6911]],

        [[-0.3916, -1.4361,  0.5472, -0.3411,  0.7598]],

        [[ 0.1138,  0.0881,  1.0145,  1.0903,  0.2113]]])
tensor([[0],
        [3],
        [5]])


tensor([-1.4361,  0.5472,  0.7598,  0.1138,  0.0881,  1.0145,  1.0903,  0.2113])

In [30]:
lengths_samples = my_visited.sum(-1).view(-1)
concat_logprobs = x[my_visited]
other_max_location = dirpg_cpp.padded_logsumexp(concat_logprobs,lengths_samples)

In [31]:
ignore = other_max_location == -np.inf

In [35]:
torch.tensor([[True], [False], [True]]) | ignore

tensor([[ True],
        [False],
        [ True]])

In [24]:
other_max_location = dirpg_cpp.padded_logsumexp(concat_logprobs,lengths_samples)

0
tensor([])
tensor(-inf)
3
tensor([-0.0211,  0.1169, -1.1231])
tensor(0.8872)
5
tensor([-0.8852,  0.1871,  0.9724, -1.2777, -0.5325])
tensor(1.6348)


In [25]:
mu = torch.randn(batch, graph_size)
gumbel_max,_= torch.max(utils_gumbel.sample_gumbel(mu),-1,True)
print(gumbel_max.size())
print(other_max_location.size())
other_max_gumbel = utils_gumbel.sample_truncated_gumbel(other_max_location,gumbel_max)

torch.Size([3, 1])
torch.Size([3, 1])


In [26]:
other_max_gumbel

tensor([[  -inf],
        [0.0889],
        [1.4026]])

In [78]:
visited = torch.tensor([(True, True, False),
                  (True, True, True),
                  (False, False, False)])

leng = torch.rand(3)
print(leng)
torch.where(torch.all(visited.t(), dim=0, keepdim=True),leng+100, leng)

tensor([0.6125, 0.4573, 0.8552])


tensor([[  0.6125, 100.4573,   0.8552]])

In [80]:
~torch.all(visited.t(), 0, keepdim=True)

tensor([[ True, False,  True]])

In [86]:
visited = torch.tensor([[True, True, False],
                  [True, True, True],
                  [False, False, False]])

In [87]:
torch.all(visited.t(), 0)

tensor([False,  True, False])

In [48]:
flag = False
a = torch.tensor([100, 100, 9])
b = torch.tensor([55, 98, 11])
print((a > b))
torch.all((a > b))

tensor([ True,  True, False])


tensor(False)

In [54]:
x = torch.randn(10,1,5)

torch.any(torch.tensor([-1,2,3], device=x.device) == -1)

tensor(True)

In [ ]:
#include "ATen/Parallel.h"
#include <torch/extension.h>
#include <iostream>
#include <vector>

//#include <pybind11/stl.h>
//#include <pybind11/complex.h>
//#include <pybind11/functional.h>

using namespace std;

class BatchedGraphs{
    BatchedGraphs(torch::Tensor distance_matrices){

    };

};
torch::Tensor padded_logsumexp(torch::Tensor concat_logprobs, torch::Tensor lengths){
    // Args: concat_logprobs: non-even length for each sample in the batch
    // length: the slices of logprobs for each sample (i.e. sum(lengths) == concat_logprobs.size())
    torch::Tensor logsumexp_res = torch::zeros({lengths.size(0),1});
    int prev_len = 0;
    for (int i=0; i<lengths.size(0);i++){
        int sample_length = lengths[i].item<int>();
        torch::Tensor sample_logprobs = torch::zeros({sample_length});
        for (int j=0; j < sample_length; j++){
            sample_logprobs[j] = concat_logprobs[prev_len+j];
        }
        logsumexp_res[i] = sample_logprobs.logsumexp({0});
        prev_len = prev_len + sample_length;
    }
    return logsumexp_res;
}


struct HeapNode {

  float priority;
  bool t_opt;
  bool done;
  py::object python_node_object;
  int idx_original;



  HeapNode(py::list node_args){
    priority = node_args[0].cast<float>();
    t_opt = node_args[1].cast<bool>();
    done = node_args[2].cast<bool>();
    python_node_object = node_args[3];
    idx_original = node_args[4].cast<int>();

    //py::list batched_prefix(python_node_object.attr("prefix"));


  }
  HeapNode(const HeapNode &other){
      priority = other.priority;
      t_opt = other.t_opt;
      python_node_object = other.python_node_object;
      idx_original = other.idx_original;
      done = other.done;
  }

  bool operator<(const HeapNode &other) const {
     if (t_opt == other.t_opt){
        return priority < other.priority;
     }
     if (t_opt && !other.t_opt){
        return false;
     }
     else{
        return true;
     }
  }

};


class NodeConvertor{
    static const batch_size = 100;
    torch::Tensor ids = torch::zeros({batch_size,1});
    torch::Tensor t = torch::zeros({batch_size,1});
    torch::Tensor next_actions = torch::zeros({batch_size,1,8});
    torch::Tensor not_visited = torch::zeros({batch_size,1,8});
    torch::Tensor prefix = torch::zeros({batch_size,8});
    torch::Tensor lengths = torch::zeros({batch_size,1});
    torch::Tensor cur_coord = torch::zeros({batch_size,1,2});
    torch::Tensor done = torch::zeros({batch_size,1});
    torch::Tensor logprob_so_far = torch::zeros({batch_size,1});
    torch::Tensor bound_togo = torch::zeros({batch_size,1});
    torch::Tensor max_gumbel = torch::zeros({batch_size,1});
    torch::Tensor is_t_opt = torch::zeros({batch_size});
     
    Convertor(py::list )
    
    torch::Tensor operator[](int i) const
};

/*
py::list stack_heap_nodes_vec(vector<HeapNode> heap_nodes){
    int batch_size = heap_nodes.size();
    torch::Tensor ids = torch::zeros({batch_size,1});
    torch::Tensor t = torch::zeros({batch_size,1});
    torch::Tensor next_actions = torch::zeros({batch_size,1,8});
    torch::Tensor not_visited = torch::zeros({batch_size,1,8});
    torch::Tensor prefix = torch::zeros({batch_size,8});
    torch::Tensor lengths = torch::zeros({batch_size,1});
    torch::Tensor cur_coord = torch::zeros({batch_size,1,2});
    torch::Tensor done = torch::zeros({batch_size,1});
    torch::Tensor logprob_so_far = torch::zeros({batch_size,1});
    torch::Tensor bound_togo = torch::zeros({batch_size,1});
    torch::Tensor max_gumbel = torch::zeros({batch_size,1});
    torch::Tensor is_t_opt = torch::zeros({batch_size});

    torch::Tensor ids_object;
    py::object t_object;
    py::object next_actions_object;
    py::object not_visited_object;
    py::object prefix_object;
    py::object lengths_object;
    py::object cur_coord_object;
    py::object done_object;
    py::object logprob_so_far_object;
    py::object bound_togo_object;
    py::object max_gumbel_object;
    py::object is_t_opt_object;
    for (int i=0; i < batch_size; i++){
        py::list temp_ids = heap_nodes[i].python_node_object.attr("ids");
        ids_object = temp_ids[heap_nodes[i].idx_original];
        ids[i] = temp_ids[heap_nodes[i].idx_original];

        //ids[i] = heap_nodes[i].python_node_object.attr("ids")[heap_nodes[i].idx_original];
        t[i]  = heap_nodes[i].python_node_object.t[heap_nodes[i].idx_original];
        next_actions[i] = heap_nodes[i].python_node_object.next_actions[heap_nodes[i].idx_original];
        not_visited[i] = heap_nodes[i].python_node_object.not_visited[heap_nodes[i].idx_original];
        prefix[i] = heap_nodes[i].python_node_object.prefix[heap_nodes[i].idx_original];
        lengths[i] = heap_nodes[i].python_node_object.lengths[heap_nodes[i].idx_original];
        cur_coord[i] = heap_nodes[i].python_node_object.cur_coord[heap_nodes[i].idx_original];
        done[i] = heap_nodes[i].python_node_object.done[heap_nodes[i].idx_original];
        logprob_so_far[i] = heap_nodes[i].python_node_object.logprob_so_far[heap_nodes[i].idx_original];
        bound_togo[i] = heap_nodes[i].python_node_object.bound_togo[heap_nodes[i].idx_original];
        max_gumbel[i] = heap_nodes[i].python_node_object.max_gumbel[heap_nodes[i].idx_original];
        is_t_opt[i] = heap_nodes[i].python_node_object.is_t_opt[heap_nodes[i].idx_original];
    }

    py::list to_return(ids, t,next_actions, not_visited, prefix, lengths, cur_coord, done, logprob_so_far,bound_togo,max_gumbel,is_t_opt);
    return to_return;
}
*/
struct Trajectory{
    int idx;
    py::object actions;  //this is actually a tensor
    float cost;
    float objective;

    Trajectory(HeapNode leaf_node){
        py::list batch_objective(leaf_node.python_node_object.attr("objective"));
        py::list batch_lengths(leaf_node.python_node_object.attr("lengths"));
        py::list batch_actions(leaf_node.python_node_object.attr("prefix"));

        idx = leaf_node.idx_original;
        objective = batch_objective[idx].cast<float>();
        cost = batch_lengths[idx].cast<float>();


        actions = batch_actions[idx];
    }
};

class Heap {
 private:
    std::vector<HeapNode> elements_;
 public:
    Heap(HeapNode new_node){
        elements_.push_back(new_node);
        push_heap(elements_.begin(), elements_.end());
    };
    void push(HeapNode new_node) {

        elements_.push_back(new_node);
        push_heap(elements_.begin(), elements_.end());
    }

    HeapNode pop() {
      HeapNode result = elements_.front();
      pop_heap(elements_.begin(), elements_.end());
      elements_.pop_back();
      return result;
    }

    py::object get_elements_(){
         vector<double> r;
         vector<bool> b;
         for (int i=0; i<elements_.size(); i++){
            r.push_back(elements_[i].priority);
            b.push_back(elements_[i].t_opt);
         }
         return py::make_tuple(py::cast(r),py::cast(b));
    };
    bool is_empty() {
        return elements_.empty();
    };
};

class BatchedHeaps{
    private:
        int batch_size_ = 100;
    public:
        vector<Heap> heaps_;

        BatchedHeaps(py::list root_nodes){
            batch_size_ = py::len(root_nodes);
            for (int sample_idx=0; sample_idx < batch_size_; sample_idx++){
                heaps_.push_back(Heap(HeapNode(root_nodes[sample_idx])));
            }
        }

        py::list pop_batch(){
            py::list heap_nodes;
            py::list trajectories;
            py::list to_return;
            for (int sample_idx=0; sample_idx < batch_size_; sample_idx++){
                bool to_pop = true; //!heaps_[sample_idx].is_empty();
                while (to_pop){
                    HeapNode node(heaps_[sample_idx].pop());
                    to_pop = node.done;
                    if(to_pop){
                        Trajectory traj(node);
                        trajectories.append(traj);
                        }
                     else{
                        heap_nodes.append(node);
                     }
                }

            }
            to_return.append(heap_nodes);
            to_return.append(trajectories);
            return to_return;
        }

        /*
        py::list pop_batch(){
            vector<HeapNode> heap_nodes;
            py::list trajectories;
            py::list to_return;
            for (int sample_idx=0; sample_idx < batch_size_; sample_idx++){
                bool to_pop = true; //!heaps_[sample_idx].is_empty();
                while (to_pop){
                    HeapNode node(heaps_[sample_idx].pop());
                    to_pop = node.done;
                    if(to_pop){
                        Trajectory traj(node);
                        trajectories.append(traj);
                        }
                     else{
                        heap_nodes.push_back(node);
                     }
                }

            }
            py::list torch_heap_node = stack_heap_nodes_vec(heap_nodes);
            to_return.append(torch_heap_node);
            to_return.append(trajectories);
            return to_return;
        }
        */
        void push_batch(py::list heap_nodes, torch::Tensor ignore_list){
            for (int sample_idx=0; sample_idx < batch_size_; sample_idx++){
                if (!ignore_list[sample_idx].item<bool>()){
                    heaps_[sample_idx].push(HeapNode(heap_nodes[sample_idx]));
                    }
            }
        }
        void push_batch(py::list heap_nodes){
            for (int sample_idx=0; sample_idx < batch_size_; sample_idx++){
                heaps_[sample_idx].push(HeapNode(heap_nodes[sample_idx]));

            }
        }
        int size(){
            return heaps_.size();
        }
};


namespace py = pybind11;
PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {

  m.def("padded_logsumexp", &padded_logsumexp, "padded logsumexp");

  py::class_<BatchedHeaps>(m, "BatchedHeaps") // returns BatchedHeapNode
      .def(py::init<py::list>())
      //.def(py::init<py::object>())
      .def("pop_batch", &BatchedHeaps::pop_batch)
      .def("push_batch",(void (BatchedHeaps::*)(py::list)) &BatchedHeaps::push_batch)
      .def("push_batch",(void (BatchedHeaps::*)(py::list, torch::Tensor)) &BatchedHeaps::push_batch)
      .def("size", &BatchedHeaps::size)
      .def_readonly("heaps_", &BatchedHeaps::heaps_);

  py::class_<HeapNode>(m, "HeapNode")
      //.def(py::init<double,double,bool,double>())
      .def(py::init<py::list>())
      .def_readwrite("python_node_object", &HeapNode::python_node_object)
      .def_readwrite("priority", &HeapNode::priority)
      .def_readwrite("done", &HeapNode::done)
      .def_readwrite("idx_original", &HeapNode::idx_original)
      .def_readwrite("t_opt", &HeapNode::t_opt);


  py::class_<Trajectory>(m, "Trajectory")
      .def(py::init<HeapNode>())
      .def_readonly("idx", &Trajectory::idx)
      .def_readonly("actions", &Trajectory::actions)
      .def_readonly("cost", &Trajectory::cost)
      .def_readonly("objective", &Trajectory::objective);

  py::class_<Heap>(m, "Heap")
      .def(py::init<HeapNode>())
      .def("get_elements_", &Heap::get_elements_)
      .def("push", &Heap::push)
      .def("pop", &Heap::pop) //
      .def("is_empty", &Heap::is_empty); //pybind11::return_value_policy::move

}


In [ ]:


PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
  //m.def("stack_heap_nodes_vec", &stack_heap_nodes_vec);
  m.def("dannys_main", &dannys_main);
  //m.def("test_args", &test_args);
  m.def("compute_mst", &compute_mst);
  m.def("fill_graphs", &fill_graphs);

  py::class_<BatchedGraphs>(m, "BatchedGraphs")
      .def(py::init<int, int>())
      .def("setWeights", &BatchedGraphs::setWeights)
      .def("mstCost",(float (BatchedGraphs::*)(int)) &BatchedGraphs::mstCost)
      .def("mstCost",(float (BatchedGraphs::*)(int, const MstNode&)) &BatchedGraphs::mstCost);
      //.def("mstCost", &BatchedGraphs::mstCost);

  py::class_<BatchedHeaps>(m, "BatchedHeaps") // returns BatchedHeapNode
      .def(py::init<vector<float>, vector<int>,vector<int>,vector<int>,vector<int>,vector<vector<int>>,vector<vector<int>>,
           vector<vector<int>>,vector<float>,vector<vector<float>>,vector<float>,vector<float>,vector<float>,py::object>()) //   py::keep_alive<1, 2>(), py::return_value_policy::take_ownership
      //.def(py::init<py::object>())
      .def("pop_batch", &BatchedHeaps::pop_batch)
      .def("push_batch", &BatchedHeaps::push_batch)
      .def("erase_heaps", &BatchedHeaps::erase_heaps)
      //.def("push_batch",(void (BatchedHeaps::*)(NodeConverter&)) &BatchedHeaps::push_batch)
      //.def("push_batch",(void (BatchedHeaps::*)(NodeConverter&, torch::Tensor)) &BatchedHeaps::push_batch)

      .def("size", &BatchedHeaps::size)
      .def_readonly("heaps_", &BatchedHeaps::heaps_);
  /*
  py::class_<HeapNode>(m, "HeapNode")
      //.def(py::init<float,float,bool,float>())
      .def(py::init<pair<py::list, NodeConverter*>>())
      .def_readwrite("python_node_object", &HeapNode::python_node_object)
      .def_readwrite("priority", &HeapNode::priority)
      .def_readwrite("done", &HeapNode::done)
      .def_readwrite("idx_original", &HeapNode::idx_original)
      .def_readwrite("t_opt", &HeapNode::t_opt);
  */

  py::class_<Trajectory>(m, "Trajectory")
      //.def(py::init<HeapNode>())
      .def_readonly("idx", &Trajectory::idx)
      .def_readonly("actions", &Trajectory::actions)
      .def_readonly("cost", &Trajectory::cost)
      .def_readonly("objective", &Trajectory::objective);


  /*
  py::class_<Heap>(m, "Heap")
      .def(py::init<HeapNode*>())
      //.def("get_elements_", &Heap::get_elements_)
      .def("push", &Heap::push)
      .def("pop", &Heap::pop) //
      .def("is_empty", &Heap::is_empty); //pybind11::return_value_policy::move
    */
  py::class_<NodeConverter>(m, "NodeConverter")
      //.def(py::init<vector<float>, vector<int>, vector<int>,vector<int>, vector<int>,vector<vector<int>>,
      //vector<vector<int>>, vector<vector<int>>, vector<float>, vector<vector<float>>, vector<float>, vector<float>,
      // vector<float>, py::object>())
      .def_readwrite("priority", &NodeConverter::priority)
      .def_readwrite("t_opt", &NodeConverter::t_opt)
      .def_readwrite("done", &NodeConverter::done)
      .def_readwrite("ids", &NodeConverter::ids)
      .def_readwrite("t", &NodeConverter::t)
      .def_readwrite("next_actions", &NodeConverter::next_actions)
      .def_readwrite("not_visited", &NodeConverter::not_visited)
      .def_readwrite("prefix", &NodeConverter::prefix)
      .def_readwrite("lengths", &NodeConverter::lengths)
      .def_readwrite("cur_coord", &NodeConverter::cur_coord)
      .def_readwrite("logprob_so_far", &NodeConverter::logprob_so_far)
      .def_readwrite("bound_togo", &NodeConverter::bound_togo)
      .def_readwrite("max_gumbel", &NodeConverter::max_gumbel);

  //py::class_<BatchedMST>(m, "BatchedMST")
      //.def(py::init<int, int>());
      //.def("split", &BatchedHeaps::pop_batch)
      //.def_readonly("idx", &BatchedMST::idx)
}


### Local

In [ ]:

----------- TIME -----------
total split_time:  4333658.0
####
split_mst_time:  117853.0
split_gumbel_time:  3267578.0
split_info_time:  109630.0
####
mst_special_time:  443371.0
mst_other_time:  314612.0
push_special_time:  0.0
push_other_time:  77474.0

In [ ]:
----------- relative TIME -----------
total split:  0.8325034320045269
####
split_mst:  0.033957322248823654
split_gumbel:  0.9334352723013422
split_info:  0.032607405449834155
####
mst_special:  0.08845562726795424
mst_other:  0.06381098933426958
push_special:  0.0
push_other:  0.015229951393249203

## Server w/o GPU

In [ ]:
----------- relative TIME -----------
total split:  0.8646668973627671
####
split_mst:  0.02277734107151524
split_gumbel:  0.9532011169044822
split_info:  0.024021542024002553
####
mst_special:  0.07139416426935402
mst_other:  0.05286502766220614
push_special:  0.0
push_other:  0.011073910705672664

In [ ]:
----------- TIME -----------
total split_time:  51906851.0
####
split_mst_time:  1384630.0
split_gumbel_time:  43318674.0
split_info_time:  1261338.0
####
mst_special_time:  4141806.0
mst_other_time:  3155809.0
push_special_time:  0.0
push_other_time:  689712.0
epoch: 0, train_batch_id: 0
avg_cost opt: 9.3265, avg_cost direct: 8.7274
candidates: 1.61  prune_count: 2.73
grad_norm: 907.0209, clipped: 1.0
epsilon:  3.0
batch time:  8.768082857131958
ETA (D:H:M) 12:23:45


In [ ]:
tensor([[[-0.0014,  0.1032, -0.1584],
         [-0.1416, -0.0741,  0.0516],
         [-0.0038,  0.1526, -0.0171]],

        [[ 0.0509, -0.0582, -0.0378],
         [-0.1839, -0.1275, -0.0793],
         [ 0.0071,  0.0761,  0.1155]],

        [[-0.1305, -0.0838,  0.0699],
         [ 0.1598, -0.0396,  0.1440],
         [-0.0310,  0.0204,  0.1743]]], grad_fn=<SelectBackward>)

In [ ]:
tensor([[[-0.0014,  0.1032, -0.1584],
         [-0.1416, -0.0741,  0.0516],
         [-0.0038,  0.1526, -0.0171]],

        [[ 0.0509, -0.0582, -0.0378],
         [-0.1839, -0.1275, -0.0793],
         [ 0.0071,  0.0761,  0.1155]],

        [[-0.1305, -0.0838,  0.0699],
         [ 0.1598, -0.0396,  0.1440],
         [-0.0310,  0.0204,  0.1743]]], grad_fn=<SelectBackward>)